In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Extraction_Data")\
        .config("spark.driver.extraClassPath", "/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar")\
        .getOrCreate()

24/10/11 19:56:42 WARN Utils: Your hostname, Ubuntu-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/11 19:56:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/11 19:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from datetime import datetime

current_year = datetime.now().year

In [3]:
type(datetime.now().year)

int

In [4]:
ano_inicial = 2023
ano_final = 2024
for ano in range(int(ano_inicial), int(ano_final) + 1):
    print(ano)

2023
2024


In [5]:
ano_inicial = current_year
ano_final = current_year

dif_anos = (ano_final - ano_inicial) + 1
for i in range(dif_anos):
    print(ano_inicial + i)

2024


In [6]:
from pyspark.sql.functions import input_file_name, transform, col, substring
from pyspark.sql.types import StringType

origin = '/home/adriano/Documentos/airflow/database/dest/bronze/execucao-financeira'
process_all = True
ano_final = current_year
ano_inicial = current_year


if process_all == False:
    origin_file = origin + '/' + str(ano_inicial)+ '.json'
    df = spark.read.json(origin_file)
elif process_all == True:
    origin_file = origin + '/*.json'
    df = spark.read.json(origin_file) 
    df = df.withColumn('nomeArquivo',substring(input_file_name(),-9,4))


#print(df.show())
print(df.printSchema())

root
 |-- autorEmenda: string (nullable = true)
 |-- codigoAmparoLegal: long (nullable = true)
 |-- descricaoEmpenho: string (nullable = true)
 |-- fonteRecurso: string (nullable = true)
 |-- idProjetoInvestimento: string (nullable = true)
 |-- informacoesComplementares: string (nullable = true)
 |-- localEntrega: string (nullable = true)
 |-- naturezaDespesa: string (nullable = true)
 |-- nomeEsferaOrcamentaria: string (nullable = true)
 |-- nomeFavorecido: string (nullable = true)
 |-- nomeTipoEmpenho: string (nullable = true)
 |-- nrPtres: string (nullable = true)
 |-- numeroNotaEmpenhoGerada: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- pagina: long (nullable = true)
 |-- planoInterno: string (nullable = true)
 |-- planoOrcamentario: string (nullable = true)
 |-- resultadoPrimario: string (nullable = true)
 |-- tamanhoDaPagina: long (nullable = true)
 |-- tipoCredito: string (nullable = true)
 |-- ugEmitente: string (nullable = true)
 |-- ugResponsave

In [8]:
import jaydebeapi
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('/home/adriano/Documentos/airflow/.env')
load_dotenv(dotenv_path=dotenv_path)

user_dw = os.getenv('USER_DW')
password_dw = os.getenv('PASSWORD_DW')
host_dw = os.getenv('HOST_DW')
database_dw = os.getenv('DATABASE_DW')
port_dw = os.getenv('PORT_DW')
driver = "org.postgresql.Driver"

mode = 'append'
url = f'jdbc:postgresql://{host_dw}:{port_dw}/{database_dw}'
properties = {"user": user_dw, "password": password_dw, "driver": driver}
conn = jaydebeapi.connect(driver, url, [user_dw, password_dw], '/home/adriano/Documentos/airflow/jdbc/postgresql-42.7.4.jar')
df.write.jdbc(url=url, table='stg_execucao_financeira', mode=mode, properties=properties)





24/10/11 19:58:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
curs = conn.cursor()
curs.execute(f'''delete from stg_execucao_financeira where "nomeArquivo" = '{ano_final}' ''')